In [0]:
df_junto = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")
df_junto.show(5)

In [0]:
df_junto.count()

In [0]:
df_junto.select("moeda").distinct().count()

In [0]:
moedas = ['USD', 'EUR', 'GBP']

df_moedas = df_junto.filter(df_junto.moeda.isin(moedas))
df_moedas.show()

In [0]:
df_moedas.select("moeda").distinct().count()

In [0]:
df_moedas.printSchema()

In [0]:
from pyspark.sql.functions import to_date

df_moedas = df_moedas.withColumn("data", to_date("data"))
df_moedas.printSchema()

In [0]:
from pyspark.sql.functions import first

resultado_taxas_conversao = df_moedas.groupBy("data") \
           .pivot("moeda") \
           .agg(first("taxa")) \
           .orderBy("data", ascending=False)

resultado_taxas_conversao.show()

In [0]:
resultado_valores_reais = resultado_taxas_conversao.select("*")
resultado_valores_reais.show(5)

In [0]:
from pyspark.sql. functions import col, round

for moeda in moedas: 
        resultado_valores_reais = resultado_valores_reais\
                                            .withColumn(
                                                moeda, round(1/col(moeda), 4)
                                            )

In [0]:
resultado_taxas_conversao = resultado_taxas_conversao.coalesce(1)
resultado_valores_reais = resultado_valores_reais.coalesce(1)

In [0]:
resultado_taxas_conversao.write\
    .mode ("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/taxas_conversao")
    
resultado_valores_reais.write\
    .mode ("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/valores_reais")

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-results/prata/valores_reais/"))

In [0]:
display(dbutils.fs.ls("dbfs:/databricks-results/prata/taxas_conversao/"))